In [ ]:
import numpy as np
import pandas as pd

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
# default value of heaad command is 5 ie 0 to 4
movies.head()
# we can give any N in the head command
movies.head(1)
credits.head()
credits.head(1)
# gives all the values present in cast column
credits.head(1)["cast"].values
credits.head(1)["crew"].values
# merging the two databases on the basis of title column -- it can be done with either title or movie id
movies = movies.merge(credits, on = "title")
# shape will give number of rows and number of columns in a tuple form
movies.shape
movies.shape
credits.shape
# removing those columns which are not needed
# number of different datas present in a particular column ie original language
movies["original_language"].value_counts()
# The information contains
# number of columns
# column labels
# column data types
# memory usage
# range index
# number of cells in each column (non-null values).
movies.info()
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]
movies.head()
# removing missing data and null data
# isnull will find the columns whose data is null
movies.isnull()
movies.isnull().sum()
# The dropna() method -- removes the rows that contains NULL values.
# inplace -- default False. If True: the removing is done on the current DataFrame. If False: returns a copy where the removing is done.
movies.dropna(inplace = True)
movies.duplicated()
# Pandas duplicated() method helps in analyzing duplicate values only. 
# It returns a boolean series which is True only for Unique elements.
movies.duplicated().sum()
# formatting all the columns
# The iloc property -- gets, or sets, the value(s) of the specified indexes [rows, columns]
movies.iloc[0].genres
# we need to convert this list of dictionaries into a list only like [action,adventure,fantasy,science fiction]
def convert(obj):
    l = []
    for i in obj:
        l.append(i["name"])
    return l
# the string indeces mut be integers
# this code will return ["action", "adventure", "fantasy", "science fiction"]
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
# since it is a string the function above is not running
# so it is first needed to convert in list
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l
movies["genres"].apply(convert)
# func: .apply takes a function and applies it to all values of pandas series
movies["genres"] = movies["genres"].apply(convert)
movies.head()
movies["keywords"]= movies["keywords"].apply(convert)
movies.head()
# needed only first 3 actors of the dictionary
movies["cast"][0]
def convert_3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i["name"])
            counter += 1
        else:
            break
    return l

movies["cast"] = movies["cast"].apply(convert_3)
movies.head(1)
movies["crew"][0]
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
            break
    return l
movies["crew"].apply(fetch_director)
movies["crew"] = movies["crew"].apply(fetch_director)
movies.head(1)
movies["overview"][0]
movies["overview"].apply(lambda x:x.split())
# string to list converter
movies["overview"] = movies["overview"].apply(lambda x:x.split())
movies.head()
# the first name of the actors may be same which might confuse the model
# thus removing the space is best method
movies["genres"].apply(lambda x:[i.replace(" ", "")for i in x])
movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ", "")for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ", "") for i in x])
movies["cast"] = movies["cast"].apply(lambda x:[i.replace(" ","")for i in x])
movies["crew"] = movies["crew"].apply(lambda x:[i.replace(" ","")for i in x])
movies["overview"] = movies["overview"].apply(lambda x:[i.replace(" ", "") for i in x])
movies.head()
# making just one tags column with all the information needed for the data
movies["tags"] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]
movies.head()
# making a new database just with the required data
new_df = movies[["movie_id", "title", "tags"]]
new_df.head()
new_df["tags"].apply(lambda x: " ".join(x))
# A lambda function is a small anonymous function.
# A lambda function can take any number of arguments, but can only have one expression.
# making strings into lists
new_df["tags"] = new_df["tags"].apply(lambda x: " ".join(x))
new_df
new_df["tags"][0]
# advised to convert it into lower case
new_df["tags"].apply(lambda x: x.lower())
new_df["tags"] = new_df["tags"].apply(lambda x: x.lower())
new_df


# data vectorization
# how can we say that these following movies are similar
# we need to find the similarity between the tags to do that we need to find the similarity between words which can be done by counting the number of times that word occur in the given tag
# we will conver the text into vectors
# in 2d space we can make vectors which will have an x coordinate and a y coordinate
# each movie will be considered as a vector
# now vectors which are closer to each other will represent same movies or similar movies
# we will use bag of wordsw vectorizatiopn technique
# all the tags will be concatenated
# 5000 most common words will be calculated 
# each tag will be checked with the the 5000 most common words list and a hashmap will be created
# less most common words means faster vectorization
# stop words will not be considered( stop words are those words which are not actually a part of the sentence or give meaning to the sentence -- and, a , but etc)

from sklearn.feature_extraction.text import CountVectorizer
# max_features = how many wordsw for hash map need to be considered
cv = CountVectorizer(max_features = 5000, stop_words = "english")
#count vectorizer will by default return sparce matrix which needs to be converted into an array
# so it will be explicedly converted to a numpy array
vectors = cv.fit_transform(new_df["tags"]).toarray()
cv.fit_transform(new_df["tags"]).toarray().shape
vectors[1]
cv.get_feature_names_out
# corpes is the big list with all the words which were combined of the tags
len(cv.get_feature_names_out())
# actions and action form two words which need to be removed
# stemming -- will remove this ambiguity and combine these words
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()  # object
def stem(text):
    y = []
    for i in text.split():
        ps.stem(i)

ps.stem("dancing, dance, danced")
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
new_df["tags"] = new_df["tags"].apply(stem)
# 4806 movies have 4806 vectors and 5000 words
# we need to calculate the distance of every mopvie with the other movies
# greater the distance less the similarity
# we will not calculate euclidian distance(tip to tip distance) we will calculate cosine distance(angular distance)
# angle in closer to zero basically same vector
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors)
similarity = cosine_similarity(vectors)
similarity.shape
# this is the similarity of the first movie with the first movie
similarity[0].shape
# diagonal of similarities will be 1
# we need to find the index of movie from the data
new_df[new_df["title"] == "Avatar"]   #.index[0]
def recommend(input):
    movie_index = new_df[new_df["title"] == input].index[0]
    distances = similarity[movie_index]
    # sorting will loose the index position of the movie
    # enumerate will be used
    # reverse order since intial values will mostly be zero
    # hey so that it can be done on the basis of second value not the first
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[0:20]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        
recommend('Batman')
# for finding the name again
new_df.iloc[1216].title
import pickle
pickle.dump(new_df,open("movies.pkl","wb"))
pickle.dump(new_df.to_dict(),open("movie_dict.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))
